In [1]:
import phyloGraph as ph

from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)

#vim ~/.Plotly/.Credentials

## Load Data

In [2]:
pgd = ph.phyloData()

the data of phylo


Pick one of the methods below for getting data

### 1) fetch from Tree of Life

In [3]:
pgd.search_name("primates")

,name,id
0,Primates,15963


In [4]:
#pgd.fetch_tol_data(15963, limit=25000)

In [5]:
#pgd.add_time()
#pgd.df.head()

### 2) load from raw file

In [6]:
#raw_file = "data/primate_raw.xml"
#pgd.load_raw_file(raw_file)

In [7]:
#pgd.write_prep_data("data/mammal_df.csv", "data/mammal_links.txt")

### 3) load from prepared files

In [8]:
#pgd.load_prep_data("data/Mammalia-15040-df.csv")
pgd.load_prep_data("data/Chordata-2499-df.csv")


### return data for plotting

In [9]:
df = pgd.return_data()
print(df.shape)
df.head()

(10395, 11)


,ancestor,depth,extinct,id,name,num_kids,phylesis,x,y,Begin,End
0,2466,1,0,2499,Chordata,2,0,-0.794184,0.918448,521.0,0.0
1,2499,2,2,14819,Calcichordata,0,0,-0.091174,0.033661,NaN,NaN
2,2499,2,0,14821,Urochordata,8,0,-1.014761,0.366714,541.0,485.4
3,14821,3,0,114489,Octacnemidae,0,0,-0.066112,-0.370996,NaN,NaN
4,14821,3,0,114490,Phlebobranchiata,4,0,-0.344893,-0.201929,NaN,NaN


## Plot Data

In [10]:
pgp = ph.phyloGraph(df)

the plotting of phylo


In [11]:
pgp.search_name('Mamma')

                 name     id
164          Mammalia  15040
1734  Mammalodontidae  16029


In [12]:
root_node=15040 # mammals
#root_node=15963 # primates

In [13]:
pgp.create_plot_df(root_node)

/Users/Seth/Documents/tolGraph/phyloGraph.py:487: RuntimeWarning:

All-NaN slice encountered



In [14]:
pgp.rejitter_XY()

In [15]:
pgp.create_plot_data()
#pgp.create_plot_data(Z_dim='depth', Z_dim_mult=-1)

Loaded plot data. Root node: 15040


In [16]:
#pgp.render_plot()

In [17]:
pgp.search_name('Eutheria')

         name     id
183  Eutheria  15997


In [18]:
#focus_node = 16024 # whales
#focus_node = 16293 # Catarrhini
#focus_node = 16299 # Hominidae
pgp.refocus(16293)

/Users/Seth/Documents/tolGraph/phyloGraph.py:564: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Loaded plot data. Highlighting: 16293


In [19]:
#pgp.render_plot(publish=True, filename="Mammals")